In [1]:
#black scholes classes

In [2]:
#initialize libraries
import numpy as np
import pandas as pd
import math
from scipy.stats import norm

In [3]:
#Black Scholes Pricing Model

#S = asset price
#K = strike price
#T = expiration time (yrs)
#r = risk free rate (annualized)
#sig = volatility

# initialize the parent class
class BlackScholes:
    def __init__(self,S,K,T,r,sig):
        self.S = S
        self.K = K
        self.T = T
        self.r = r
        self.sig = sig

        #derived factors
        # d1 shows how far the current price is above the strike price, considering growth and time.
        # d2 indicates the likelihood of the option expiring in the money.
        self.d1 = (np.log(self.S/self.K) + (self.r + (self.sig**2 / 2))*self.T)/(self.sig*math.sqrt(self.T))
        self.d2 = self.d1 - self.sig*math.sqrt(self.T)
    
    #creating functions for normal pdf and cdf in case we want to edit distribution
    def calcCDF(self,dTerm): 
        return norm.cdf(dTerm)

    def calcPDF(self,dTerm): 
        return norm.pdf(dTerm)
    
    # greek calculations that are the same for both put/calls below

    def calcGamma(self):
        return self.calcPDF(self.d1) / (self.S * self.sig * math.sqrt(self.T))

    def calcVega(self):
        return self.S * self.calcPDF(self.d1) * math.sqrt(self.T)
    
    #print function

    def print(self):
        print("The option has the following outputs: ")
        print("Price: ${:.4f}".format(self.calcPrice()))
        print("Delta: ${:.4f}".format(self.calcDelta()))
        print("Gamma: ${:.4f}".format(self.calcGamma()))
        print("Vega: ${:.4f}".format(self.calcVega()))
        print("Theta: ${:.4f}".format(self.calcTheta()))
        print("Rho: ${:.4f}".format(self.calcRho()))
        print()


In [5]:
#child class for call options
class BSCall(BlackScholes):
    def __init__(self,S,K,T,r,sig):
        super().__init__(S,K,T,r,sig)

    def calcPrice(self):
        return self.S * self.calcCDF(self.d1) - self.K *  (math.exp(-self.r * self.T)) * self.calcCDF(self.d2)

    def calcDelta(self):
        return self.calcCDF(self.d1)
    
    def calcTheta(self):
        return - (self.S * self.calcPDF(self.d1)*self.sig)/(2*math.sqrt(self.T)) - (self.r * self.K * math.exp(-self.r * self.T) * self.calcCDF(self.d2))
    
    def calcRho(self):
        return self.K * self.T * math.exp(-self.r * self.T) * self.calcCDF(self.d2)


#child class for put options
class BSPut(BlackScholes):
    def __init__(self,S,K,T,r,sig):
        super().__init__(S,K,T,r,sig)

    def calcPrice(self):
        return self.K *  (math.exp(-self.r * self.T)) * self.calcCDF(-self.d2) - self.S * self.calcCDF(-self.d1)
    
    def calcDelta(self):
        return self.calcCDF(self.d1) - 1
    
    def calcTheta(self):
        return - (self.S * self.calcPDF(self.d1) * self.sig) / (2 * math.sqrt(self.T))+ (self.r * self.K * math.exp(-self.r * self.T) * self.calcCDF(-self.d2))
 
    def calcRho(self):
        return -self.K * self.T * math.exp(-self.r * self.T) * self.calcCDF(-self.d2)
